### Initialization

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle

import sys
sys.path.append('../common')
from utils import process_group

fpath1 = '/home/8ididata/2021-2/babnigg202107_2/cluster_results_QZ'
fpath2 ='/home/8ididata/2022-1/babnigg202203/cluster_results_QZ'


### Averaging of SA-XPCS results, temperature ramp up

In [2]:
avg_ramp_up, t_el, ql_dyn, ql_sta = process_group(group='T214',
                                              num_sections=20,
                                              prefix=fpath1, 
                                              zone_idx=1,
                                              num_cores=24)

2023-12-20 16:48:47,830 utils                   : total number of files in T214  is 2390
2023-12-20 16:50:05,751 xpcs_viewer.fileIO.hdf_reader: key not found: /xpcs/sphilist
2023-12-20 16:50:05,759 utils                   : failed to read file /home/8ididata/2021-2/babnigg202107_2/cluster_results_QZ/T214_H04_2nd_500C10p_att00_Rq0_01075_0001-100000.hdf, skip this file
2023-12-20 16:50:08,503 utils                   : label='T214_section_01': remove 13 datasets out of 120
2023-12-20 16:50:08,524 utils                   : label='T214_section_00': remove 13 datasets out of 120
2023-12-20 16:50:08,530 utils                   : label='T214_section_03': remove 13 datasets out of 120
2023-12-20 16:50:08,577 utils                   : label='T214_section_04': remove 14 datasets out of 120
2023-12-20 16:50:08,571 utils                   : label='T214_section_02': remove 16 datasets out of 120
2023-12-20 16:50:08,738 utils                   : label='T214_section_08': remove 13 datasets out of 119


### Averaging of SA-XPCS results, temperature ramp down

In [3]:
avg_ramp_down, t_el, ql_dyn, ql_sta = process_group(group='T215',
                                              num_sections=20,
                                              prefix=fpath1, 
                                              zone_idx=1,
                                              num_cores=24)

2023-12-20 16:50:10,528 utils                   : total number of files in T215  is 2390
2023-12-20 16:50:17,363 utils                   : label='T215_section_01': remove 11 datasets out of 120
2023-12-20 16:50:17,419 utils                   : label='T215_section_00': remove 13 datasets out of 120
2023-12-20 16:50:17,424 utils                   : label='T215_section_02': remove 12 datasets out of 120
2023-12-20 16:50:17,527 utils                   : label='T215_section_03': remove 14 datasets out of 120
2023-12-20 16:50:17,578 utils                   : label='T215_section_04': remove 12 datasets out of 120
2023-12-20 16:50:17,693 utils                   : label='T215_section_06': remove 15 datasets out of 120
2023-12-20 16:50:17,714 utils                   : label='T215_section_08': remove 15 datasets out of 120
2023-12-20 16:50:17,714 utils                   : label='T215_section_05': remove 14 datasets out of 120
2023-12-20 16:50:17,717 utils                   : label='T215_section_0

### Averaging of background 

In [4]:

avg_bg, _, _, _ = process_group(group='D029',
                                 num_sections=1, 
                                 prefix=fpath1,
                                 zone_idx='auto',
                                 num_cores=24)


2023-12-20 16:50:19,457 utils                   : total number of files in D029  is 200
2023-12-20 16:50:23,883 utils                   : label='D029_section_00': remove 26 datasets out of 200


### Data reduction

Subtracting I(Q) with background, converting to absolute scattering cross-section, and dump to pickle file for plotting

In [5]:

fname = 'E0110_S3_4_Ramp_340C10p_att00_Rq0_00001_0001-100000.hdf'

with h5py.File(fpath2+'/'+fname, 'r') as f:
    _ = f['measurement']['sample']['absolute_cross_section_scale'][()]
    abs_coeff = _[0][0]

for ii in range(len(avg_ramp_up)):
    avg_ramp_up[ii]["saxs_1d"] = abs_coeff*(avg_ramp_up[ii]["saxs_1d"] - avg_bg[0]["saxs_1d"])

for ii in range(len(avg_ramp_down)):
    avg_ramp_down[ii]["saxs_1d"] = abs_coeff*(avg_ramp_down[ii]["saxs_1d"] - avg_bg[0]["saxs_1d"])


with open('checkpoint.pickle', 'wb') as f:  # 'wb' for binary write mode
    pickle.dump(avg_ramp_up, f)  # Dump the object to the file
    pickle.dump(avg_ramp_down, f)
    pickle.dump(t_el, f)
    pickle.dump(ql_sta, f)
    pickle.dump(ql_dyn, f)
